In [1]:

from ipywidgets import widgets
from IPython.display import display


In [4]:
%cd ../

/Users/adam/Dropbox/CanvasCorrections


# Setup the configuration

## Canvas API Token

Instructions go here....

In [6]:
from assets.Configuration import InteractiveConfiguration
from assets.InputFields import make_course_ids_input, make_canvas_token_input, make_general_reset_button

In [7]:
make_canvas_token_input()

Text(value='', description='Canvas token')

In [9]:
# You may confirm that the token has been added to the configuration here
InteractiveConfiguration.canvas_token

'5fguu'

## Course ids

Instructions go here

In [10]:
make_course_ids_input()

# Figure out what needs grading

In [ ]:
print("These assignments need grading: ")
for sec in InteractiveConfiguration.course_ids:
    to_grade = get_assignments_needing_grading(sec)
    print(sec, to_grade)

# Double check the automatic assignments

In [ ]:


class Manager(object):
    def __init__(self):
        self.results = [] 

In [ ]:
def make_consolidated_text_file(journal_folder, filename=COMPILED_TEXT_FILE_NAME):
    """Writes each entry in to a single text file for ease of reading"""
    entry = """
    {student_id} \n
    {body} \n
    \n
    """
    to_write = "%s/%s" %(journal_folder, filename) 
    with open("%s/all-submissions.json" % journal_folder, 'r') as f:
        j = json.load(f)
        with open(to_write, 'w+') as cf:
            for rec in j:
            #     print(entry.format(**rec))
                cf.write(entry.format(**rec))
    print("Consolidated file written to {}".format(to_write))

In [ ]:
def make_text_display(student_id, text):
    """Handles the formatting of the student's text"""
    entry = """
          <div id='%s'>
            <h3>%s</h3>
            <p>
                %s
            </p>
        </div>"""
    e = entry % (student_id, student_id, text)
    return widgets.HTML(e)

def make_submission_output(text, student_id, credit_list):
    bval = 'Credit' if student_id in credit_list else 'No credit'
    credit_button = widgets.ToggleButtons(
    options=['Credit', 'No credit'],
        value=bval,
    description='',
    disabled=False
    )
    
    ta = make_text_display(student_id, text)

    def on_value_change(change):
        """The relevant callback
        NB the use of scope to define student_id"""
        v = change['new']
        print(v, student_id)
        try:
            if v == 'Credit':
                credit_list.append(student_id)
            elif v == 'No credit':
                credit_list.remove(student_id)
        except ValueError:
            pass

    credit_button.observe(on_value_change, names='value')
    display(widgets.VBox([ta, credit_button]))

def make_consolidated_text_fields(results, credit_list=[]):
    """Displays each entry with a toggle to adjust whether the 
    student receives credit"""
    for r in results:
        make_submission_output(r['body'], r['student_id'], credit_list)

In [ ]:
test_results = [{'student_id' : i, 'body': 'Body text for student {}'.format(i)} for i in range(0,4)]

In [ ]:
a = [1, 'b', 4]
# a.remove('b')
a

In [ ]:
'Credit' if 2 in [1, 2] else 'No credit'


In [ ]:
test_results

credit_list = [0, 2]

make_consolidated_text_fields(test_results, credit_list)

In [ ]:
credit_list

# Upload the grades to canvas

In [ ]:


def make_upload_button(credit_list):
    def upload_callback(event):
        print(credit_list)

    b = widgets.Button(description='Upload grades', button_style='danger')
    b.on_click(upload_callback)
    display(b)

In [ ]:
make_upload_button(credit_list)